## Below is the code required to re-create Figure 2
### Topic: Making HR-diagrams from isochrones of with different evolution models; Creating J - Kp isochrones using different extinction laws

In [1]:
# Import necessary packages. 
from popstar import synthetic, evolution, atmospheres, reddening, ifmr
from popstar.imf import imf, multiplicity
import numpy as np
import pylab as py
import pdb
import os
import pylab as py

/Users/mwhosek/anaconda3/envs/astroconda2/lib/python3.6/site-packages/pysynphot/locations.py:346: UserWarning: Extinction files not found in /Users/mwhosek/models/models_2019_07/cdbs/extinction
  warnings.warn('Extinction files not found in %s' % (extdir, ))


In [2]:
# Define isochrone parameters
logAge = 6.70 # Age in log(years)
AKs = 1.0 # Ks filter extinction in mags
dist = 4000 # distance in parsecs
metallicity = 0 # metallicity in [M/H]
atm_func = atmospheres.get_merged_atmosphere

# Define evolution models
evo_merged = evolution.MergedBaraffePisaEkstromParsec()
evo_mist = evolution.MISTv1(version=1.2) # Version 1.2 is the default
evo_parsec = evolution.Parsec()

# Define 3 extinction laws to explore: Cardelli+89, Nishiyama+09, Hosek+19
redlaw = reddening.RedLawCardelli(3.1) # Rv = 3.1
redlaw2 = reddening.RedLawNishiyama09()
redlaw3 = reddening.RedLawHosek18b()
    
# Also specify filters for synthetic photometry (optional). Here we use 
# the HST WFC3-IR F127M, F139M, and F153M filters
filt_list = ['nirc2,J', 'nirc2,Kp']

# Define iso directories for each evolution model. This is important because
# otherwise the isochrones will overwrite each other (evolution model is
# not included in part of file name).
iso_dir_merged = 'iso_merged_r1/'
iso_dir_mist = 'iso_mist_r1/'
iso_dir_parsec = 'iso_parsec_r1/'

# Make isochrone directories, if they don't already exist
arr = [iso_dir_merged, iso_dir_mist, iso_dir_parsec]
for ii in arr:
    if not os.path.exists(ii):
        os.mkdir(ii)

In [3]:
# Make Isochrone objects. Note that is calculation will take a few minutes per 
# isochrone, unless they have been made already.
# To speed up isochrone generation, we will set mass sampling = 3,
# which says to generate every 3rd point that exists on the original
# model isochrone
iso_merged = synthetic.IsochronePhot(logAge, AKs, dist, metallicity=metallicity,
                                 evo_model=evo_merged, atm_func=atm_func,
                                 filters=filt_list, red_law=redlaw,
                                 iso_dir=iso_dir_merged, mass_sampling=3)

iso_mist = synthetic.IsochronePhot(logAge, AKs, dist, metallicity=metallicity,
                                 evo_model=evo_mist, atm_func=atm_func,
                                  filters=filt_list, red_law=redlaw,
                                 iso_dir=iso_dir_mist, mass_sampling=3)
    
iso_parsec = synthetic.IsochronePhot(logAge, AKs, dist, metallicity=metallicity,
                                 evo_model=evo_parsec, atm_func=atm_func,
                                 filters=filt_list, red_law=redlaw,
                                 iso_dir=iso_dir_parsec, mass_sampling=3)


In [4]:
# We'll also explore the effects of using different extinction laws, but 
# with the same total exitinction. Generate MIST isochrones for the other
# extinction laws already defined. Put these in a separate iso_dir
iso_dir_mist_r2 = 'iso_mist_r2/'
iso_dir_mist_r3 = 'iso_mist_r3/'

for ii in [iso_dir_mist_r2, iso_dir_mist_r3]:
    if not os.path.exists(ii):
        os.mkdir(ii)

iso_mist_r2 = synthetic.IsochronePhot(logAge, AKs, dist, metallicity=metallicity,
                                 evo_model=evo_mist, atm_func=atm_func,
                                  filters=filt_list, red_law=redlaw2,
                                 iso_dir=iso_dir_mist_r2, mass_sampling=3)

iso_mist_r3 = synthetic.IsochronePhot(logAge, AKs, dist, metallicity=metallicity,
                                 evo_model=evo_mist, atm_func=atm_func,
                                  filters=filt_list, red_law=redlaw3,
                                 iso_dir=iso_dir_mist_r3, mass_sampling=3)

In [5]:
# Make 2-panel plot showing the impact of the choice of evolution models on the HR-diagram
# and the choice of extinction law on the CMD

# HRD model plot
py.figure(figsize=(20,10))
py.subplots_adjust(left=0.1)
py.subplot(121)
py.plot(np.log10(iso_merged.points['Teff']), np.log10(iso_merged.points['L']), 'b-',
                label='Merged', linewidth=2)
py.plot(np.log10(iso_mist.points['Teff']), np.log10(iso_mist.points['L']), 'k-',
                label='MIST', linewidth=2)
py.plot(np.log10(iso_parsec.points['Teff']), np.log10(iso_parsec.points['L']), 'r-',
                label='Parsec', linewidth=2)
py.xlabel('log(Teff) [K]', fontsize=24)
py.ylabel('log(L) [W]', fontsize=24)
py.gca().invert_xaxis()
py.tick_params(axis='both', labelsize=20)
py.legend(loc=3, fontsize=22)
py.title('Changing Evolution Models', fontsize=28)

# CMD Extinction plot
py.subplot(122)
py.plot(iso_mist.points['m_nirc2_J'] - iso_mist.points['m_nirc2_Kp'], 
        iso_mist.points['m_nirc2_J'], 'b-', label='C89', linewidth=2)
py.plot(iso_mist_r2.points['m_nirc2_J'] - iso_mist_r2.points['m_nirc2_Kp'], 
        iso_mist_r2.points['m_nirc2_J'], 'k-', label='N09', linewidth=2)
py.plot(iso_mist_r3.points['m_nirc2_J'] - iso_mist_r3.points['m_nirc2_Kp'], 
        iso_mist_r3.points['m_nirc2_J'], 'r-', label='H19', linewidth=2)
py.xlabel('NIRC2 J - Kp (mag)', fontsize=24)
py.ylabel('NIRC2 J (mag)', fontsize=24)
py.gca().invert_yaxis()
py.tick_params(axis='both', labelsize=20)
py.legend(loc=3, fontsize=22)
py.title('Changing Exinction Laws', fontsize=28)
py.savefig('iso_comp.png')